## Data structures

In [ ]:
from __future__ import annotations
from dataclasses import dataclass
from functools import reduce
from typing import List, Tuple
from multimethod import multimethod
import math

# One Qbits class holding any number of qubits it's enough actually

@dataclass
class Qbits:
    vector : List[float] #column vector
    # tensor product: multiply each element of another array by
    # all the elements of this array
    @multimethod
    def __mul__(self, q: Qbits) -> Qbits:
        v = []
        for x in q.vector:
            for y in self.vector:
                v.append(x * y)
        return Qbits(v)
    def __len__(self) -> int:
        return len(self.vector)
    @multimethod
    def __mul__(self, s: float) -> Qbits:
        return Qbits([s * x for x in self.vector])
    @multimethod
    def __mul__(self, s: int) -> Qbits:
        return self.__mul__(float(s))
    @multimethod
    def __mul__(self, _):
        return NotImplemented
    @multimethod
    def __rmul__(self, s: float) -> Qbits:
        return self.__mul__(s)
    @multimethod
    def __rmul__(self, s: int) -> Qbits:
        return self.__rmul__(float(s))
    
@dataclass(frozen=True)
class Qbit:
    vector : List[float] #column vector
    @multimethod
    def __mul__(self, q: Qbit) -> Qbits:
        v = [0.] * 4
        v[0] = self.vector[0] * q.vector[0]
        v[1] = self.vector[1] * q.vector[0]
        v[2] = self.vector[0] * q.vector[1]
        v[3] = self.vector[1] * q.vector[1]
        return Qbits(v)
    @multimethod
    def __mul__(self, s: float) -> Qbit:
        return Qbit([s * self.vector[0], s * self.vector[1]])
    @multimethod
    def __mul__(self, s: int) -> Qbit:
        return self.__mul__(float(s))
    def __add__(self, q: Qbit) -> Qbit:
        return Qbit([self.vector[0] + q.vector[0], self.vector[1] + q.vector[1]])
    @multimethod
    def __rmul__(self, q: Qbits) -> Qbits:
        v = []
        for x in q.vector:
            v.append(x * self.vector[0])
        for x in q.vector:    
            v.append(x * self.vector[1])
        return Qbits(v)
    @multimethod
    def __rmul__(self, s: float) -> Qbit:
        return self.__mul__(s)
    @multimethod
    def __rmul__(self, s: int) -> Qbit:
        return self.__rmul__(float(s))
    def __getitem__(self, i: int) -> float:
        return self.vector[i]

@dataclass
class Gate:
    matrix : List[List[float]]
    # 2x2 Matrix x  2 d column vector
    @multimethod
    def __mul__(self, q: Qbit) -> Qbit:
        return Qbit([self.matrix[0][0] * q.vector[0] + self.matrix[0][1] * q.vector[1], 
                     self.matrix[1][0] * q.vector[0] + self.matrix[1][1] * q.vector[1]])
    # n x n matrix * 1 x n column vector
    @multimethod
    def __mul__(self, q: Qbits) -> Qbits:
        v = [0.] * len(self.matrix)
        for r in range(len(self.matrix)):
            for c in range(len(self.matrix[0])):
                v[r] += self.matrix[r][c] * q.vector[c]
        return Qbits(v)

def is_zero(n: float) -> bool:
    return abs(n) < 1e-9

QONE = Qbit([0.,1.])
QZERO = Qbit([1.,0])
QNULL = Qbit([0., 0.])

## Functions

In [ ]:
def zero() -> Qbit:
    return Qbit([1,0])

def one() -> Qbit:
    return Qbit([0,1])

def qbits(t: str) -> Qbits | Qbit:
    #least significant bit to the right
    t = t[::-1]
    v = []
    for b in t:
        if b == '1':
            v.append(1)
        elif b == '0':
            v.append(0)
        else:
            raise Exception(f"Wrong qbit format '{b}'")
    if len(v) == 2:
        return Qbit(v)
    else:
        return Qbits(v)

def entangled_qbits(t: str) -> Qbits | Qbit:
    v = []
    #least significant bit to the right
    t = t[::-1]
    for b in t:
        if b == '1':
            v.append(one())
        elif b == '0':
            v.append(zero())
        else:
            raise Exception(f"Wrong qbit format '{b}'")
    if len(v) == 1:
        return v[0]
    else:
        return reduce(lambda x, y: x * y, v)

def ntobin(n: int, digits: int) -> [float]:
    v = []
    while n:
        v.append(float(n % 2))
        n = n // 2
    return v + [0] * (digits-len(v)) if digits > 0 else v

def qbit(d: int) -> Qbit:
    if d == 0:
        return zero()
    elif d == 1:
        return one()
    else:
        raise Exception(f"Invalid number '{d}'")

def zero() -> Qbit:
    return Qbit([1,0])

def one() -> Qbit:
    return Qbit([0,1])

def qbits(t: str) -> Qbits | Qbit:
    #least significant bit to the right
    t = t[::-1]
    v = []
    for b in t:
        if b == '1':
            v.append(1)
        elif b == '0':
            v.append(0)
        else:
            raise Exception(f"Wrong qbit format '{b}'")
    if len(v) == 2:
        return Qbit(v)
    else:
        return Qbits(v)

def disentangle(q: Qbits) -> [Qbits]:
    N = len(q.vector)
    print(q)
    QN = int(math.log2(N))
    print(N)
    v : List[Qbits] = []
    for i in range(N):
        v.append(q.vector[i] * Qbits(ntobin(i, QN)))
    return v

def bits_to_qbits(n, num) -> List[Qbit]:
    b = ntobin(n, num)
    return [qbit(int(i)) for i in b]

def bits_to_bits(n, num) -> List[Qbit]:
    b = ntobin(n, num)
    return [int(i) for i in b]

def measure_bits(q: Qbits) -> List[Tuple[float, List[int]]]:
    return [(e, bits_to_bits(i, int(math.log2(len(q))))) for (i, e) in enumerate(q.vector) if abs(e) > 1e-9]

def measure_prob_bits(q: Qbits) -> List[Tuple[float, List[int]]]:
    return [(e*e, bits_to_bits(i, int(math.log2(len(q))))) for (i, e) in enumerate(q.vector) if abs(e) > 1e-9]

def measure_qbits(q: Qbits) -> List[Tuple[float, List[Qbit]]]:
    return [(e, bits_to_qbits(i, int(math.log2(len(q))))) for (i, e) in enumerate(q.vector)]
    
    

## Gates

In [ ]:
import math

def H(q: Qbit) -> Qbit:
    sq = 1/math.sqrt(2)
    return Gate([[sq,  sq],
                 [sq, -sq]]) * q
def X(q: Qbit) -> Qbit:
    return Gate([[0,1],[1,0]]) * q

def Y(q: Qbit) -> Qbit:
    return Gate([[0,-1j],[1j,0]]) * q

def Z(q: Qbit) -> Qbit:
    return Gate([[1, 0], [0,-1]])

def S(q: Qbit) -> Qbit:
    return Gate([[1,0], [0, 1j]])

def T(q: Qbit) -> Qbit:
    return Gate([[1,0], 
                 [0, math.cos(math.pi/4) + 1j * math.sin(math.pi/4)]])

def CZ(q: Qbits) -> Qbits:
    return Gate([[1,0,0,0],
                 [0,1,0,0],
                 [0,0,1,0],
                 [0,0,0,-1]]) * q

def SWAP(q: Qbits) -> Qbits:
    return Gate([[1,0,0,0],
                 [0,0,1,0],
                 [0,1,0,0],
                 [0,0,0,1]]) * q

def CNOT(q: Qbits) -> Qbits:
    return Gate([[1,0,0,0],
                 [0,1,0,0],
                 [0,0,0,1],
                 [0,0,1,0]]) * q

def Toffoli(q: Qbits) -> Qbits:
    return Gate([[1, 0, 0, 0, 0, 0, 0, 0],
                 [0, 1, 0, 0, 0, 0, 0, 0],
                 [0, 0, 1, 0, 0, 0, 0, 0],
                 [0, 0, 0, 1, 0, 0, 0, 0],
                 [0, 0, 0, 0, 1, 0, 0, 0],
                 [0, 0, 0, 0, 0, 1, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 1],
                 [0, 0, 0, 0, 0, 0, 1, 0]]) * q

CCNOT = Toffoli

## Output

Compute the output qubit values given an array of entangled qubit combinations

$$
\frac{1}{\sqrt{2}} \ket{010} - \frac{1}{\sqrt{2}} \ket{110} \rightarrow 
\begin{pmatrix}
\frac{1}{\sqrt{2}} \\
- \frac{1}{\sqrt{2}}
\end{pmatrix}
\begin{pmatrix}
0 \\
1
\end{pmatrix}
\begin{pmatrix}
1 \\
0
\end{pmatrix}
$$

The most significant qubit to the left has a 50% chance of appearing in the output while the other two are the same in all cases and do have a 100% of appearing in the output.

In [ ]:
@dataclass
class QbitWeight:
    zero : Tuple[float, Qbit] = (0., QZERO)
    one:   Tuple[float, Qbit] = (0., QONE)
    
def qbit_output(q: List[Qbits]) -> List[QbitWeight]:
    # remvove zero-probability qubit combinations
    m = [(x,c) for (x,c) in measure_qbits(q) if abs(x) > 1e-9] # Array of Arrays of Qbit elements
    #[(0.7071067811865475, [Qbit(vector=[1, 0]), Qbit(vector=[0, 1]), Qbit(vector=[1, 0])]), (-0.7071067811865475, [Qbit(vector=[0, 1]), Qbit(vector=[0, 1]), Qbit(vector=[1, 0])])]
    NQBITS = len(m[0][1])
    QBIT_COMBINATIONS = len(m)
    WEIGHT_IDX = 0
    QBIT_COMB_IDX = 1
    weighted_qbits : List[QbitWeight] = [QbitWeight] * NQBITS
    # iterate over qubits
    for q in range(NQBITS):
        # iterate over qubit combinations
        for c in range(QBIT_COMBINATIONS):
            #select qubit q from qubit combination c 
            qc = m[c][QBIT_COMB_IDX][q]
            #select the weight of qubit combination c
            w = m[c][WEIGHT_IDX]
            wq = weighted_qbits[q]
            if qc == QZERO:
                x = w + wq.zero[0]
                weighted_qbits[q] = QbitWeight((x, QZERO), wq.one)
            elif qc == QONE:
                x = w + wq.one[0]
                weighted_qbits[q] = QbitWeight(wq.zero, (x, QONE))
            else:
                raise Exception(f"Invalid Qubit state {qc}") 
    # It can happen that +/- coefficients cancel each other out
    # in which case the Qbit has a 100% chance of being present in the output
    # since the probability is the sum of the coefficient squared.
    # For zero coefficients simply assign 100% (1) to the qubit found in the matching position.
    # Note that the zero-probability qubits have been remvoed in the first step of the algorithms
    # and therefore ALL qbits do have a weight greater than zero i.e. are present in the output
    for (i,e) in enumerate(weighted_qbits):
        if e.zero[0] == 0. and e.one[0] == 0.:
            # check the value of the qbit at position i in qubit array
            y = m[0][QBIT_COMB_IDX][i]
            if y == QZERO:
                weighted_qbits[i].zero = (1.0, QZERO)
            elif y == QONE:
                weighted_qbits[i].one = (1.0, QONE)
            else:
                raise Exception(f"Invalid qubit value: {other}")
      
    return weighted_qbits

In [ ]:
c1 = H(zero())
c2 = one()
print(c1*c2)
print(disentangle(c1*c2))

### Circuit representation.

Matrix:
 * num column = max number of stages
 * num rows = number of wires / qubits
 * element = gate with wire info e.g. H(wire1,wire2,...)
 * null element = id: output = input
```
[[H(0),  CNOT(0,1), H(0) ],
 [id(1), id(1),     id(1)]] 
```

### Simulation algorithm

1. disentangle --> retrieve all qbit possible states
2. if not last stage:
    1. compute all possible input states from output
    2. for each input state generate all possible output
    3. got to 1

In [ ]:
t = H(QONE) * QONE * QZERO
t

In [ ]:
measure_prob_bits(t)

In [ ]:
m = measure_bits(t)
m

In [ ]:
[ [w * i for i in x] for (w, x) in m]

In [ ]:
q = measure_qbits(t)
q

In [ ]:
import pprint
pp = pprint.PrettyPrinter()
g = [ [w * i for i in x] for (w, x) in q]
pp.pprint(g)

In [ ]:
from functools import reduce
from operator import add
M = [[w * i for i in x] for (w, x) in m]
[reduce(add, [M[i][j] for i in range(len(M))]) for j in range(len(M[0]))]


In [ ]:
import random
@dataclass
class QbitWeight:
    zero : Tuple[float, Qbit] = (0., QZERO)
    one:   Tuple[float, Qbit] = (0., QONE)
    
def qbit_output(q: List[Qbits]) -> List[QbitWeight]:
    # remvove zero-probability qubit combinations
    m = [(x,c) for (x,c) in measure_qbits(q) if abs(x) > 1e-9] # Array of Arrays of Qbit elements
    #[(0.7071067811865475, [Qbit(vector=[1, 0]), Qbit(vector=[0, 1]), Qbit(vector=[1, 0])]), (-0.7071067811865475, [Qbit(vector=[0, 1]), Qbit(vector=[0, 1]), Qbit(vector=[1, 0])])]
    NQBITS = len(m[0][1])
    QBIT_COMBINATIONS = len(m)
    WEIGHT_IDX = 0
    QBIT_COMB_IDX = 1
    weighted_qbits : List[QbitWeight] = [QbitWeight] * NQBITS
    # iterate over qubits
    for q in range(NQBITS):
        # iterate over qubit combinations
        for c in range(QBIT_COMBINATIONS):
            #select qubit q from qubit combination c 
            qc = m[c][QBIT_COMB_IDX][q]
            #select the weight of qubit combination c
            w = m[c][WEIGHT_IDX]
            wq = weighted_qbits[q]
            if qc == QZERO:
                x = w + wq.zero[0]
                weighted_qbits[q] = QbitWeight((x, QZERO), wq.one)
            elif qc == QONE:
                x = w + wq.one[0]
                weighted_qbits[q] = QbitWeight(wq.zero, (x, QONE))
            else:
                raise Exception(f"Invalid Qubit state {qc}") 
    # It can happen that +/- coefficients cancel each other out
    # in which case the Qbit has a 100% chance of being present in the output
    # since the probability is the sum of the coefficient squared.
    # For zero coefficients simply assign 100% (1) to the qubit found in the matching position.
    # Note that the zero-probability qubits have been remvoed in the first step of the algorithms
    # and therefore ALL qbits do have a weight greater than zero i.e. are present in the output
    for (i,e) in enumerate(weighted_qbits):
        if e.zero[0] == 0. and e.one[0] == 0.:
            # check the value of the qbit at position i in qubit array
            y = m[0][QBIT_COMB_IDX][i]
            if y == QZERO:
                weighted_qbits[i].zero = (1.0, QZERO)
            elif y == QONE:
                weighted_qbits[i].one = (1.0, QONE)
            else:
                raise Exception(f"Invalid qubit value: {other}")
      
    return weighted_qbits

def combine_qbits(q: List[QbitWeight]) -> List[Qbit]:
    # return linear combination of individual qbits
    return [b.zero[0] * b.zero[1] + b.one[0] * b.one[1] for b in out]

# return bit value depending on probability
def measure_output(q: List[QbitWeight]) -> List[int]:
    return [random.choices([0,1], weights=[i.zero[0] * i.zero[0], 1. - i.zero[0] * i.zero[0]])[0] for i in q]

# return samples results as string array
def sample_output_str(q: List[Qbits], num_samples: int) -> List[str]:
    return ["".join([str(j) for j in measure_output(qbit_output(q))]) for _ in range(num_samples)]

In [ ]:
qbit_output(t)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from collections import Counter
sampled_output = sample_output_str(t, num_samples=1000)#["".join([str(j) for j in measure_output(qbit_output(t))]) for _ in range(1000)]
c = Counter(sampled_output)
print(c)
plt.bar(sampled_output, c.most_common()[0][1])

In [ ]:
out = qbit_output(t)
combine_qbits(out)